This notebook contains an example for teaching.

## Introduction

In labor economics an important question is what determines the wage of workers. This is a causal question,
but we could begin to investigate from a predictive perspective.

In the following wage example, $Y$ is the hourly wage of a worker and $X$ is a vector of worker's characteristics, e.g., education, experience, gender. Two main questions here are:


* How to use job-relevant characteristics, such as education and experience, to best predict wages?

* What is the difference in predicted wages between men and women with the same job-relevant characteristics?

In this lab, we focus on the prediction question first.

## Data


The data set we consider is from the March Supplement of the U.S. Current Population Survey, year 2015.  We select white non-hispanic individuals, aged 25 to 64 years, and working more than 35 hours per week during at least 50 weeks of the year. We exclude self-employed workers; individuals living in group quarters; individuals in the military, agricultural or private household sectors;  individuals with inconsistent reports on earnings and employment status; individuals with allocated or missing information in any of the variables used in the analysis; and individuals with hourly wage below $3$. 

The variable of interest $Y$ is the hourly wage rate constructed as the ratio of the annual earnings to the total number of hours worked, which is constructed in turn as the product of number of weeks worked and the usual number of hours worked per week. In our analysis, we also focus on single (never married) workers. The final sample is of size $n=5150$.

## Data analysis

We start by loading the data set.

In [154]:
load("wage2015_subsample_inference.Rdata")
dim(data)

[1] 5150   20

Let's have a look at the structure of the data.

In [155]:
str(data)

'data.frame':	5150 obs. of  20 variables:
 $ wage : num  9.62 48.08 11.06 13.94 28.85 ...
 $ lwage: num  2.26 3.87 2.4 2.63 3.36 ...
 $ sex  : num  1 0 0 1 1 1 1 0 1 1 ...
 $ shs  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ hsg  : num  0 0 1 0 0 0 1 1 1 0 ...
 $ scl  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ clg  : num  1 1 0 0 1 1 0 0 0 1 ...
 $ ad   : num  0 0 0 1 0 0 0 0 0 0 ...
 $ mw   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ so   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ we   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ ne   : num  1 1 1 1 1 1 1 1 1 1 ...
 $ exp1 : num  7 31 18 25 22 1 42 37 31 4 ...
 $ exp2 : num  0.49 9.61 3.24 6.25 4.84 ...
 $ exp3 : num  0.343 29.791 5.832 15.625 10.648 ...
 $ exp4 : num  0.24 92.35 10.5 39.06 23.43 ...
 $ occ  : Factor w/ 369 levels "10","20","40",..: 159 136 269 23 99 86 226 232 184 146 ...
 $ occ2 : Factor w/ 22 levels "1","2","3","4",..: 11 10 19 1 6 5 17 17 13 10 ...
 $ ind  : Factor w/ 236 levels "370","380","390",..: 204 117 12 165 231 176 171 135 210 201 ...
 $ ind2 : Factor w/ 

We are constructing the output variable $Y$ and the matrix $Z$ which includes the characteristics of workers that are given in the data.

In [156]:
Y <- log(data$wage)
n <- length(Y)
Z <- data[-which(colnames(data) %in% c("wage","lwage"))]
p <- dim(Z)[2]

cat("Number of observation:", n, '\n')
cat( "Number of raw regressors:", p)

Number of observation: 5150 
Number of raw regressors: 18

For the outcome variable *wage* and a subset of the raw regressors, we calculate the empirical mean to get familiar with the data.

In [157]:
library(xtable)
Z_subset <- data[which(colnames(data) %in% c("lwage","sex","shs","hsg","scl","clg","ad","mw","so","we","ne","exp1"))]
table <- matrix(0, 12, 1)
table[1:12,1]   <- as.numeric(lapply(Z_subset,mean))
rownames(table) <- c("Log Wage","Sex","Some High School","High School Graduate","Some College","College Graduate", "Advanced Degree","Midwest","South","West","Northeast","Experience")
colnames(table) <- c("Sample mean")
tab<- xtable(table, digits = 2)
tab

,Sample mean
,<dbl>
Log Wage,2.97078670
Sex,0.44446602
Some High School,0.02330097
High School Graduate,0.24388350
Some College,0.27805825
College Graduate,0.31766990
Advanced Degree,0.13708738
Midwest,0.25961165
South,0.29650485


E.g., the share of female workers in our sample is ~44% ($sex=1$ if female).

Alternatively, we can also print the table as latex.

In [158]:
print(tab, type="latex")

% latex table generated in R 4.0.4 by xtable 1.8-4 package
% Sun Mar 07 23:11:00 2021
\begin{table}[ht]
\centering
\begin{tabular}{rr}
  \hline
 & Sample mean \\ 
  \hline
Log Wage & 2.97 \\ 
  Sex & 0.44 \\ 
  Some High School & 0.02 \\ 
  High School Graduate & 0.24 \\ 
  Some College & 0.28 \\ 
  College Graduate & 0.32 \\ 
  Advanced Degree & 0.14 \\ 
  Midwest & 0.26 \\ 
  South & 0.30 \\ 
  West & 0.22 \\ 
  Northeast & 0.23 \\ 
  Experience & 13.76 \\ 
   \hline
\end{tabular}
\end{table}


## Prediction Question

Now, we will construct a prediction rule for hourly wage $Y$, which depends linearly on job-relevant characteristics $X$:

\begin{equation}\label{decompose}
Y = \beta'X+ \epsilon.
\end{equation}

Our goals are

* Predict wages  using various characteristics of workers.

* Assess the predictive performance using the (adjusted) sample MSE, the (adjusted) sample $R^2$ and the out-of-sample MSE and $R^2$.


We employ two different specifications for prediction:


1. Basic Model:   $X$ consists of a set of raw regressors (e.g. gender, experience, education indicators,  occupation and industry indicators, regional indicators).


2. Flexible Model:  $X$ consists of all raw regressors from the basic model plus occupation and industry indicators, transformations (e.g., ${exp}^2$ and ${exp}^3$) and additional two-way interactions of polynomial in experience with other regressors. An example of a regressor created through a two-way interaction is *experience* times the indicator of having a *college degree*.

Using the **Flexible Model**, enables us to approximate the real relationship by a
 more complex regression model and therefore to reduce the bias. The **Flexible Model** increases the range of potential shapes of the estimated regression function. In general, flexible models often deliver good prediction accuracy but give models which are harder to interpret.

Now, let us fit both models to our data by running ordinary least squares (ols):

In [159]:
# 1. basic model
basic <- lwage~ (sex + exp1 + shs + hsg+ scl + clg + mw + so + we +occ2+ind2)
regbasic <- lm(basic, data=data)
regbasic # estimated coefficients
cat( "Number of regressors in the basic model:",length(regbasic$coef), '\n') # number of regressors in the Basic Model



Call:
lm(formula = basic, data = data)

Coefficients:
(Intercept)          sex         exp1          shs          hsg          scl  
   3.722235    -0.072857     0.008568    -0.592798    -0.504337    -0.411994  
        clg           mw           so           we        occ22        occ23  
  -0.182216    -0.027541    -0.034454     0.017249    -0.076472    -0.034678  
      occ24        occ25        occ26        occ27        occ28        occ29  
  -0.096202    -0.187915    -0.414933    -0.045987    -0.377847    -0.215752  
     occ210       occ211       occ212       occ213       occ214       occ215  
  -0.010623    -0.455834    -0.307589    -0.361440    -0.499495    -0.464482  
     occ216       occ217       occ218       occ219       occ220       occ221  
  -0.233715    -0.412588    -0.340418    -0.241480    -0.212628    -0.288413  
     occ222        ind23        ind24        ind25        ind26        ind27  
  -0.422394    -0.116836    -0.244493    -0.273533    -0.249368    -0.139588

Number of regressors in the basic model: 51 


##### Note that the basic model consists of $51$ regressors.

In [160]:
# 2. flexible model
flex <- lwage ~ sex + shs+hsg+scl+clg+occ2+ind2+mw+so+we + (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+occ2+ind2+mw+so+we)
regflex <- lm(flex, data=data)
regflex # estimated coefficients
cat( "Number of regressors in the flexible model:",length(regflex$coef)) # number of regressors in the Flexible Model


Call:
lm(formula = flex, data = data)

Coefficients:
(Intercept)          sex          shs          hsg          scl          clg  
  3.8602606   -0.0695532   -0.1233089   -0.5289024   -0.2920581   -0.0411641  
      occ22        occ23        occ24        occ25        occ26        occ27  
  0.1613397    0.2101514    0.0708570   -0.3960076   -0.2310611    0.3147249  
      occ28        occ29       occ210       occ211       occ212       occ213  
 -0.1875417   -0.3390270    0.0209545   -0.6424177   -0.0674774   -0.2329781  
     occ214       occ215       occ216       occ217       occ218       occ219  
  0.2562009   -0.1938585   -0.0551256   -0.4156093   -0.4822168   -0.2579412  
     occ220       occ221       occ222        ind23        ind24        ind25  
 -0.3010203   -0.4271811   -0.8694527   -1.2473654   -0.0948281   -0.5293860  
      ind26        ind27        ind28        ind29       ind210       ind211  
 -0.6221688   -0.5047497   -0.7295442   -0.8025334   -0.5805840   -0.9852350 

Number of regressors in the flexible model: 246

Note that the flexible model consists of $246$ regressors.

Try Lasso next

In [161]:
library(hdm)
flex <- lwage ~ sex + shs+hsg+scl+clg+occ2+ind2+mw+so+we + (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+occ2+ind2+mw+so+we)
lassoreg<- rlasso(flex, data=data)

sumlasso<- summary(lassoreg)



Call:
rlasso.formula(formula = flex, data = data)

Post-Lasso Estimation:  TRUE 

Total number of variables: 245
Number of selected variables: 24 

Residuals: 
     Min       1Q   Median       3Q      Max 
-2.03159 -0.29132 -0.01137  0.28472  3.63651 

            Estimate
(Intercept)    3.137
sex            0.000
shs           -0.480
hsg           -0.404
scl           -0.306
clg            0.000
occ22          0.041
occ23          0.109
occ24          0.000
occ25          0.000
occ26         -0.327
occ27          0.000
occ28         -0.257
occ29          0.000
occ210         0.038
occ211        -0.398
occ212         0.000
occ213        -0.131
occ214        -0.189
occ215        -0.387
occ216         0.000
occ217        -0.280
occ218         0.000
occ219         0.000
occ220         0.000
occ221         0.000
occ222        -0.209
ind23          0.000
ind24          0.000
ind25          0.000
ind26          0.000
ind27          0.000
ind28          0.000
ind29         -0.169
ind210     

Now, we can evaluate the performance of both models based on the (adjusted) $R^2_{sample}$ and the (adjusted) $MSE_{sample}$:

In [126]:
# Assess the predictive performance

sumbasic <- summary(regbasic)
sumflex <- summary(regflex)

#  R-squared 
R2.1 <- sumbasic$r.squared
cat("R-squared for the basic model: ", R2.1, "\n")
R2.adj1 <- sumbasic$adj.r.squared
cat("adjusted R-squared for the basic model: ", R2.adj1, "\n")

R2.2 <- sumflex$r.squared
cat("R-squared for the flexible model: ", R2.2, "\n")
R2.adj2 <- sumflex$adj.r.squared
cat("adjusted R-squared for the flexible model: ", R2.adj2, "\n")

R2.L <- sumlasso$r.squared
cat("R-squared for the lasso with flexible model: ", R2.L, "\n")
R2.adjL <- sumlasso$adj.r.squared
cat("adjusted R-squared for the flexible model: ", R2.adjL, "\n")

R-squared for the basic model:  0.3100465 
adjusted R-squared for the basic model:  0.3032809 
R-squared for the flexible model:  0.3511099 
adjusted R-squared for the flexible model:  0.3186919 
R-squared for the lasso with flexible model:  0.2778653 
adjusted R-squared for the flexible model:  0.2744836 


In [127]:
# calculating the MSE
MSE1 <- mean(sumbasic$res^2)
cat("MSE for the basic model: ", MSE1, "\n")
p1 <- sumbasic$df[1] # number of regressors
MSE.adj1 <- (n/(n-p1))*MSE1
cat("adjusted MSE for the basic model: ", MSE.adj1, "\n")

MSE2 <-mean(sumflex$res^2)
cat("MSE for the flexible model: ", MSE2, "\n")
p2 <- sumflex$df[1]
MSE.adj2 <- (n/(n-p2))*MSE2
cat("adjusted MSE for the flexible model: ", MSE.adj2, "\n")


MSEL <-mean(sumlasso$res^2)
cat("MSE for the lasso flexible model: ", MSEL, "\n")
pL <- length(sumlasso$coef)
MSE.adjL <- (n/(n-pL))*MSEL
cat("adjusted MSE for the lasso flexible model: ", MSE.adjL, "\n")

MSE for the basic model:  0.2244251 
adjusted MSE for the basic model:  0.2266697 
MSE for the flexible model:  0.2110681 
adjusted MSE for the flexible model:  0.221656 
MSE for the lasso flexible model:  0.2348928 
adjusted MSE for the lasso flexible model:  0.2466758 


In [108]:
library(xtable)
table <- matrix(0, 3, 5)
table[1,1:5]   <- c(p1,R2.1,MSE1,R2.adj1,MSE.adj1)
table[2,1:5]   <- c(p2,R2.2,MSE2,R2.adj2,MSE.adj2)
table[3,1:5]   <- c(pL,R2.L,MSEL,R2.adjL,MSE.adjL)
colnames(table)<- c("p","$R^2_{sample}$","$MSE_{sample}$","$R^2_{adjusted}$", "$MSE_{adjusted}$")
rownames(table)<- c("basic reg","flexible reg", "lasso flex")
tab<- xtable(table, digits =c(0,0,2,2,2,2))
print(tab,type="latex") # type="latex" for printing table in LaTeX
tab

% latex table generated in R 4.0.4 by xtable 1.8-4 package
% Sun Mar 07 18:00:16 2021
\begin{table}[ht]
\centering
\begin{tabular}{rrrrrr}
  \hline
 & p & \$R\verb|^|2\_\{sample\}\$ & \$MSE\_\{sample\}\$ & \$R\verb|^|2\_\{adjusted\}\$ & \$MSE\_\{adjusted\}\$ \\ 
  \hline
basic reg & 51 & 0.31 & 0.22 & 0.30 & 0.23 \\ 
  flexible reg & 246 & 0.35 & 0.21 & 0.32 & 0.22 \\ 
  lasso flex & 246 & 0.28 & 0.23 & 0.27 & 0.25 \\ 
   \hline
\end{tabular}
\end{table}


,p,$R^2_{sample}$,$MSE_{sample}$,$R^2_{adjusted}$,$MSE_{adjusted}$
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
basic reg,51,0.3100465,0.2244251,0.3032809,0.2266697
flexible reg,246,0.3511099,0.2110681,0.3186919,0.2216560
lasso flex,246,0.2778653,0.2348928,0.2744836,0.2466758


Considering all measures above, the flexible model performs slightly better than the basic model. 

One procedure to circumvent this issue is to use **data splitting** that is described and applied in the following.

## Data Splitting

Measure the prediction quality of the two models via data splitting:

- Randomly split the data into one training sample and one testing sample. Here we just use a simple method (stratified splitting is a more sophiscticated version of splitting that we can consider).
- Use the training sample for estimating the parameters of the Basic Model and the Flexible Model.
- Use the testing sample for evaluation. Predict the $\mathtt{wage}$  of every observation in the testing sample based on the estimated parameters in the training sample.
- Calculate the Mean Squared Prediction Error $MSE_{test}$ based on the testing sample for both prediction models. 

In [130]:
#splitting the data

set.seed(1) # to make the results replicable (generating random numbers)
random_2 <- sample(1:n, floor(n*4/5))
# draw (4/5)*n random numbers from 1 to n without replacing them
train <- data[random,] # training sample
test <- data[-random,] # testing sample
dim(train)

[1] 4120   20

In [131]:
# basic model
# estimating the parameters in the training sample
regbasic <- lm(basic, data=train)
regbasic


Call:
lm(formula = basic, data = train)

Coefficients:
(Intercept)          sex         exp1          shs          hsg          scl  
   3.641716    -0.059065     0.008635    -0.570657    -0.508393    -0.405932  
        clg           mw           so           we        occ22        occ23  
  -0.178154    -0.044486    -0.051111     0.004004    -0.083481    -0.036320  
      occ24        occ25        occ26        occ27        occ28        occ29  
  -0.091457    -0.126383    -0.416548    -0.046615    -0.385957    -0.220534  
     occ210       occ211       occ212       occ213       occ214       occ215  
  -0.030423    -0.460487    -0.317680    -0.375180    -0.465495    -0.494731  
     occ216       occ217       occ218       occ219       occ220       occ221  
  -0.212026    -0.413355    -0.329054    -0.263839    -0.241109    -0.293004  
     occ222        ind23        ind24        ind25        ind26        ind27  
  -0.410747    -0.075902    -0.121251    -0.206172    -0.151398    -0.08174

In [132]:
# calculating the out-of-sample MSE
trainregbasic <- predict(regbasic, newdata=test)
trainregbasic

12       44       71       84      129      149      191      221 
3.395851 2.734493 3.151807 2.506919 2.368192 3.173461 3.065637 3.043757 
     248      264      281      368      464      467      496      540 
2.699975 2.398635 3.080034 2.542424 2.700733 2.973891 2.940246 2.796404 
     546      576      629      687      705      765      769      809 
2.793806 2.907308 3.213768 3.222378 2.758879 2.772328 2.753855 3.005369 
     938      945      952      960     1003     1057     1059     1065 
2.502601 2.941366 3.166449 2.846778 3.070972 2.629048 3.036431 2.881320 
    1075     1076     1079     1101     1105     1143     1161     1195 
3.414267 3.504898 2.283221 2.564178 2.737425 2.761703 3.606691 2.556301 
    1224     1271     1299     1342     1387     1407     1508     1568 
2.861144 2.749369 2.809817 3.680845 3.445996 2.621132 2.826924 2.877508 
    1593     1646     1669     1693     1711     1715     1748     1749 
3.011676 2.775702 2.923326 2.811011 2.844195 3.037463 3.038273 2.898368 
    1751     1763     1782     1862     1889     1896     1915     1916 
2.913956 3.447804 2.987807 3.308917 3.448159 3.032423 2.639366 2.710743 
    1917     1930     1933     1939     1945     1946     1964     1978 
3.388704 3.639454 3.560323 3.261379 2.991757 3.476764 2.574404 3.409102 
    1999     2020     2055     2122     2132     2135     2194     2215 
2.815364 2.804909 2.796349 3.052717 2.760399 2.834512 3.066740 3.066740 
    2224     2278     2315     2330     2333     2362     2381     2465 
2.996740 3.497591 3.208588 3.172049 3.137275 2.977112 2.911219 2.936761 
    2488     2490     2500     2502     2555     2566     2592     2593 
3.107351 2.515200 2.884473 2.834005 3.120641 3.248789 3.190732 2.996836 
    2604     2635     2660     2746     2851     2879     2919     2972 
3.362746 3.364898 3.195416 2.850228 3.155803 2.618133 2.600422 3.536249 
    2990     3005     3020     3021     3103     3117     3195     3269 
2.486495 2.965091 2.784081 2.659487 3.183660 3.126963 3.537607 3.375366 
    3313     3357     3383     3428     3501     3531     3575     3605 
2.800045 3.108703 2.707201 3.030192 3.530804 3.056041 3.475381 2.868124 
    3630     3634     3641     3738     3803     3810     3819     3824 
2.802191 2.785823 2.978776 2.885989 3.312690 3.053702 3.262039 2.792172 
    3853     3855     3858     3867     3975     3996     4008     4012 
2.929577 2.837561 3.453031 2.637223 3.295419 3.283195 3.190034 3.255441 
    4015     4021     4027     4034     4042     4046     4056     4057 
3.513137 2.909294 3.791820 3.371755 3.487255 3.382399 3.146749 3.341796 
    4068     4078     4088     4095     4108     4109     4110     4111 
3.130255 3.083729 3.136610 3.373162 3.350788 3.342153 3.049343 3.701652 
    4119     4161     4180     4189     4190     4311     4318     4416 
2.852830 3.050451 2.661820 2.655253 3.127026 3.167800 2.660548 3.436540 
    4476     4492     4495     4585     4627     4658     4668     4735 
2.915678 3.440465 2.554050 3.351613 3.121649 2.702883 2.576908 2.956567 
    4752     4801     4805     4806     4837     4862     4865     4869 
2.586882 3.185483 3.288241 3.562156 3.517912 3.426356 3.174408 2.709436 
    4873     4981     5013     5021     5026     5034     5043     5069 
2.973516 2.988795 2.473201 2.692698 3.187168 3.096083 3.258672 2.824127 
    5087     5115     5117     5145     5177     5187     5207     5210 
3.081478 2.752523 2.596189 3.173009 2.658040 2.897046 2.965355 2.732098 
    5232     5288     5343     5345     5347     5348     5444     5455 
3.498372 3.170164 2.911181 3.308917 3.203620 2.998361 2.655253 2.806754 
    5487     5548     5617     5654     5667     5699     5766     5779 
3.177714 2.600207 2.648163 2.536779 3.101771 3.257564 2.673927 3.191729 
    5793     5798     5830     5889     5892     5902     5919     5946 
3.111683 2.895035 2.890652 2.694029 3.010846 3.283855 2.762490 3.055507 
    6002     6005     6034     6096     6132     6152     6210  

In [133]:
y.test <- log(test$wage)
MSE.test1 <- sum((y.test-trainregbasic)^2)/length(y.test)
R2.test1<- 1- MSE.test1/var(y.test)

cat("Test MSE for the basic model: ", MSE.test1, " ")

cat("Test R2 for the basic model: ", R2.test1)

Test MSE for the basic model:  0.1971044  Test R2 for the basic model:  0.3279559

In the basic model, the $MSE_{test}$ is quite closed to the $MSE_{sample}$.

In [143]:
# flexible model
# estimating the parameters
#options(warn=-1)
regflex <- lm(flex, data=train)

# calculating the out-of-sample MSE
trainregflex<- predict(regflex, newdata=test)
y.test <- log(test$wage)
MSE.test2 <- sum((y.test-trainregflex)^2)/length(y.test)
R2.test2<- 1- MSE.test2/var(y.test)

cat("Test MSE for the flexible model: ", MSE.test2, " ")

cat("Test R2 for the flexible model: ", R2.test2)

Test MSE for the flexible model:  0.2064107  Test R2 for the flexible model:  0.2962252

In [148]:
length(y.test)

[1] 1030

In the flexible model, the discrepancy between the $MSE_{test}$ and the $MSE_{sample}$ is not large.

It is worth to notice that the $MSE_{test}$ vary across different data splits. Hence, it is a good idea average the out-of-sample MSE over different data splits to get valid results.

Nevertheless, we observe that, based on the out-of-sample $MSE$, the basic model using ols regression performs is about as well (or slightly better) than the flexible model. 


Next, let us use lasso regression in the flexible model instead of ols regression. Lasso (*least absolute shrinkage and selection operator*) is a penalized regression method that can be used to reduce the complexity of a regression model when the number of regressors $p$ is relatively large in relation to $n$. 

Note that the out-of-sample $MSE$ on the test sample can be computed for any other black-box prediction method as well. Thus, let us finally compare the performance of lasso regression in the flexible model to ols regression.

In [115]:
# flexible model using lasso

# estimating the parameters
library(hdm)
reglasso <- rlasso(flex, data=train, post=FALSE)

# calculating the out-of-sample MSE
trainreglasso<- predict(reglasso, newdata=test)
MSE.lasso <- sum((y.test-trainreglasso)^2)/length(y.test)
R2.lasso<- 1- MSE.lasso/var(y.test)


cat("Test MSE for the lasso on flexible model: ", MSE.lasso, " ")

cat("Test R2 for the lasso flexible model: ", R2.lasso)

Test MSE for the lasso on flexible model:  0.212698  Test R2 for the lasso flexible model:  0.2747882

Finally, let us summarize the results:

In [149]:
table2 <- matrix(0, 3,2)
table2[1,1]   <- MSE.test1
table2[2,1]   <- MSE.test2
table2[3,1]   <- MSE.lasso
table2[1,2]   <- R2.test1
table2[2,2]   <- R2.test2
table2[3,2]   <- R2.lasso

rownames(table2)<- c("basic reg","flexible reg","lasso regression")
colnames(table2)<- c("$MSE_{test}$", "$R^2_{test}$")
tab2 <- xtable(table2, digits =3)
tab2

,$MSE_{test}$,$R^2_{test}$
,<dbl>,<dbl>
basic reg,0.1971044,0.3279559
flexible reg,0.2064107,0.2962252
lasso regression,0.2126980,0.2747882


In [151]:
print(tab2,type="latex") # type="latex" for printing table in LaTeX

% latex table generated in R 4.0.4 by xtable 1.8-4 package
% Sun Mar 07 23:04:23 2021
\begin{table}[ht]
\centering
\begin{tabular}{rrr}
  \hline
 & \$MSE\_\{test\}\$ & \$R\verb|^|2\_\{test\}\$ \\ 
  \hline
basic reg & 0.197 & 0.328 \\ 
  flexible reg & 0.206 & 0.296 \\ 
  lasso regression & 0.213 & 0.275 \\ 
   \hline
\end{tabular}
\end{table}
